In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
import pandas as pd
from ngsolve import grad as oldgrad

In [7]:
beta = (2,0.001)
eps = 0.01

p = lambda x: x + (exp(beta[0]*(x-1)/eps)-exp(-beta[0]/eps))/(exp(-beta[0]/eps)-1)
q = lambda y: y + (exp(beta[1]*(y-1)/eps)-exp(-beta[1]/eps))/(exp(-beta[1]/eps)-1)

exact = p(x) * q(y)

coeff =  beta[1] * p(x) +  beta[0] * q(y)

coeff_x = p(x)
coeff_y = q(y)


new_config = {
    'method': 'dg', #options are dg, hdg
    'order': [1, 2, 3],
    'beta': (2,0.001),
    'mesh_size': [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313],
    'epsilon': 0.01,
    'exact': exact,
    'coeff': coeff,
    'coeff_x': coeff_x,
    'coeff_y': coeff_y,
    'alpha': 20, 
    'enrich': True, 
}


In [8]:
class EnrichmentProxy(CoefficientFunction):
    """
    Provide wrappers for grad/Other and multiplication of enrichment lists.
    """
    def __init__(self, func, enr_list):
        self.func = func
        self.enr_list = enr_list
        self.grad_list = [CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) for coeff in self.enr_list ]
    def __call__(self):
        return self.func[0] + sum([self.func[i]*self.enr_list[i-1] for i in range(1,len(self.enr_list)+1)])
    def __mul__(self, other):
        if type(other) == EnrichmentProxy:
            return self() * other()
        else:
            return self() * other
    def __rmul__(self,other):
        return self.__mul__(other)    
    
    def __add__(self, other):
        if type(other) == EnrichmentProxy:
            return self() + other()
        else:
            return self() + other
    def __radd__(self,other):
        return self.__add__(other)
    
    def __sub__(self, other):
        if type(other) == EnrichmentProxy:
            return self() - other()
        else:
            return self() - other
    def __rsub__(self,other):
        return self.__sub__(other)
    
    def grad(self):
        mygrad = oldgrad(self.func[0])
        for i in range(1,len(self.enr_list)):
            mygrad += self.func[i] * self.grad_list[i-1]
            mygrad += oldgrad(self.func[i])*self.enr_list[i-1]
        return mygrad
    
    def Other(self):
        return EnrichmentProxy([f.Other() for f in self.func],self.enr_list)
    
def grad(q):
    if type(q) == EnrichmentProxy:
        return q.grad()
    else:
        return oldgrad(q)

    
'''Mark elements in the mesh to be enriched'''
def mark_element(Q, mesh, axes, mesh_size):
    ba = BitArray(Q.ndof)        
    ba.Clear()
    for el in Q.Elements():
        mark = False
        for v in el.vertices:
            if (mesh[v].point[axes] > 1-(0.001 * mesh_size)):
                mark = True
        for dof in el.dofs:
            ba[dof] = mark
        Qx = Compress(Q, active_dofs=ba)     
    return Qx

In [9]:
"""
(Hybrid) Discontinuous Galerkin Method with Upwinding
"""

class Convection_Diffusion():
    
    config = {
    }
    
    def __init__(self, new_config={}):
        self.config.update(new_config)        
    
    def _solveDG(self):
        for order in self.config['order']:
            for size in self.config['mesh_size']:
                mesh = Mesh(unit_square.GenerateMesh(maxh=size))
                V = L2(mesh, order=order, dgjumps=True)        

                if self.config['enrich'] == True:
                    print('Running DG enrichment option')
                    Q = L2(mesh, order=0) 
                    axes = (0, 1)
                    Qx = mark_element(Q, mesh, axes[0], size)
                    Qy = mark_element(Q, mesh, axes[1], size)

                    fes = FESpace([V, Qx, Qy], dgjumps = True)

                    u = EnrichmentProxy(fes.TrialFunction(),[self.config['coeff_x'], self.config['coeff_y']])
                    v = EnrichmentProxy(fes.TestFunction(), [self.config['coeff_x'], self.config['coeff_y']])

                else:
                    fes = V
                    u, v = fes.TnT()

                jump_u = u-u.Other()
                jump_v = v-v.Other()
                n = specialcf.normal(2)
                mean_dudn = 0.5 * n * (grad(u) + grad(u.Other()))
                mean_dvdn = 0.5 * n * (grad(v) + grad(v.Other()))

                h = specialcf.mesh_size

                # diffusion
                diffusion = grad(u) * grad(v) * dx \
                    +self.config['alpha'] * order ** 2/ h * jump_u * jump_v * dx(skeleton=True) \
                    +(-mean_dudn * jump_v - mean_dvdn * jump_u) * dx(skeleton=True) \
                    +self.config['alpha'] * order ** 2/h * u * v * ds(skeleton=True) \
                    + (-n * grad(u) * v -n * grad(v) * u) * ds(skeleton=True)

                # convection
                b = CoefficientFunction((self.config['beta'][0], self.config['beta'][1]))

                if self.config['enrich'] == True:
                    uup = IfPos(b * n, u(), u.Other()())
                else:
                    uup = IfPos(b * n, u, u.Other())

                convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dx(skeleton=True)

                acd = BilinearForm(fes)
                acd += self.config['epsilon'] * diffusion + convection
                acd.Assemble()

                # rhs
                f = LinearForm(fes)
                f += SymbolicLFI(self.config['coeff'] * v)
                f.Assemble()

                gfu = GridFunction(fes, name="uDG")        
                gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec

                error = sqrt (Integrate ((gfu-self.config['exact'])*(gfu-self.config['exact']), mesh))
                print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)

        return error
    
    
    def _solveHDG(self):
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        
        if self.config['enrich'] == True:    
            V = L2(mesh, order=order)
            F = FacetFESpace(mesh, order=order, dirichlet="bottom|left|right|top")
            fes = FESpace([V,F])
            u,uhat = fes.TrialFunction()
            v,vhat = fes.TestFunction()

            jump_u = u-uhat
            jump_v = v-vhat

            condense = True
            h = specialcf.mesh_size
            n = specialcf.normal(mesh.dim)

            dS = dx(element_boundary=True)

            # diffusion
            diffusion = grad(u) * grad(v) * dx + \
                alpha * order ** 2/h * jump_u * jump_v * dS + \
                (-grad(u) * n * jump_v - grad(v) * n * jump_u) * dS

            # convection            
            b = CoefficientFunction((beta[0],beta[1])) 
            uup = IfPos(b * n, u, uhat)
            convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dS

            acd = BilinearForm(fes, condense=condense)
            acd += eps * diffusion + convection
            acd.Assemble()

            #rhs
            f = LinearForm(fes)
            f += SymbolicLFI(coeff * v)
            f.Assemble()

            gfu = GridFunction(fes)    
            if not condense:
                inv = acd.mat.Inverse(fes.FreeDofs(), "umfpack")
                gfu.vec.data = inv * f.vec
            else:
                f.vec.data += acd.harmonic_extension_trans * f.vec

                inv = acd.mat.Inverse(fes.FreeDofs(True), "umfpack")
                gfu.vec.data = inv * f.vec

                gfu.vec.data += acd.harmonic_extension * gfu.vec
                gfu.vec.data += acd.inner_solve * f.vec

        return gfu   

In [10]:
CT = Convection_Diffusion(new_config)
CT._solveDG()

running enrichment option


NgException: don't know my dimension, space is CompoundFESpace